# Lab4.5 Emotional classification with Llama3

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

Through this notebook, you will annotate a text with Llama version 3, a Generative Large Language Model model released by [Meta](https://llama.meta.com/llama3/). 

You will read a conversation and send the utterances to the Llama server to annotate each using instructions, soca-lled prompting. For running a server locally see the notebook **how-to-install-llama-server.ipynb* from the start of the course. Note that the server needs to run in another terminally in parallel to this notebook. In case you cannot run the server, use the credentials for the CLTL server.

The code for the annotator is given in **llama_annotator.py**. It is an OpenAI client that sends a prompt request to a llama server server for a response.

### Installation of an OpenAI client

To run the annotator, you first need to install the OpenAI client using the following command line. If you ran the code for Llama Chat client you already did this.

In [9]:
#! pip install openai

Once succesfully installed, you can comment out the previous cell and you do not need to do this again when running this notebook. The OpenAI module is now installed on your machine and can be imported. The import will be done by the **llama_annotator.py** script, which we will load next.

## Creating the Llama chatbot

In [1]:
from llama_annotator import LlamaAnnotator

If there are no error messages, we can create a chatbot instance of a LlamaAnnotator as defined in **llama_annotator.py**. We define **annotator** as an instance of a LLamaCAnnotator, where we can specify three additional parameters: the *url* of the server (either local or online), the labels that we want to use for the annoation and optionally examples of the input and output.

In [2]:
### Labels to try
sentiment_labels = ["positive", "negative", "neutral"]
ekman_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
examples = [{"Input": "I love dogs", "Output": "joy"}, {"Input": "I hate cats", "Output": "disgust"}]

annotator = LlamaAnnotator(url="http://localhost:9001/v1", labels=ekman_labels, examples=examples)

My instructions are: [{'role': 'system', 'content': 'You are an intelligent assistant.'}, {'role': 'system', 'content': 'You will receive an utterance from a conversation as Input in text format.'}, {'role': 'system', 'content': "You need to determine the emotion expressed in the utterance and respond with one of the following labels:['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']"}, {'role': 'system', 'content': 'Do not output anything else.'}, {'role': 'system', 'content': 'Here are a few examples:'}, {'role': 'user', 'content': 'I love dogs'}, {'role': 'system', 'content': 'joy'}, {'role': 'user', 'content': 'I hate cats'}, {'role': 'system', 'content': 'disgust'}]


We are now going to read the conversation that we had before with Llama with our annotations and send these to the server again to annotate.

In [3]:
import pandas as pd
file = open("../lab3.machine_learning/data/iaa/annotator_Piek_human_Piek_chat_with_llama.json")
df = pd.read_json(file)
df.head()

,utterance,speaker,turn_id,Gold,Annotator
0,You are an intelligent assistant and your name...,Llama,1,neutral,auto
1,Piek,Human,2,neutral,Piek
2,"Nice to meet you, Piek! I'm Llama, your friend...",Llama,3,neutral,auto
3,I am lonely,Piek,4,sadness,Piek
4,"Piek, it sounds like you're feeling a bit down...",Llama,5,neutral,auto


From the Pandas dataframe, we can select the "utterance" column as the list of utterances and give this to the ```anotate``` funciton that is defined for our LlamaAnotator. We store the output in the annotations list.

In [4]:
#ChatCompletionChunk(id='chatcmpl-0cbeface-f64f-42b7-a3d5-f0bb64f2e8e1', 
#choices=[Choice(delta=ChoiceDelta(content=None, 
#function_call=None, refusal=None, role='assistant', tool_calls=None), 
#finish_reason=None, index=0, logprobs=None)], created=1727957746, 
#model='local-model', object='chat.completion.chunk', 
#service_tier=None, system_fingerprint=None, usage=None)

def annotate(input):
        annotations = []
        for text in input:
            ### history is reset after every turn
            annotator._history = annotator._instruct
            annotator._history.append({"role": "user", "content": "{}".format(text)})
            ### We call the openai client with a low temperature for the first turn
            completion = annotator._client.chat.completions.create(
                model="local-model", # this field is currently unused
                messages=annotator._history,
                temperature=0.0,
                stream=True,
            )
            response = ""
            for chunk in completion:
                #print(chunk) 
                if 'choices' in chunk  and chunk.choices[0].delta.content:
                    response += chunk.choices[0].delta.content
            annotations.append({"Input": text, "Output": response})
            print(text, 'response:', response)
        return annotations

In [6]:
utterances = df["utterance"]
annotations = annotate(utterances)
print(annotations)

You are an intelligent assistant and your name is Llama. I'd like to know who you're talking to! What's your name? response: 
Piek response: 
Nice to meet you, Piek! I'm Llama, your friendly AI companion. It's great to have you as a conversational partner. Is there anything you'd like to chat about or ask me for help with? response: 


RemoteProtocolError: peer closed connection without sending complete message body (incomplete chunked read)

We can add the predictions from Llama to the dataframe but Llama does not always precisely follow the instructions e.g. "Output: Output:neutral". We therefore need to clean and filter the output. Specifically, we check if any of the Ekman labels is a substring of the Llama output and then take that value. If none of these is matched, we set the value to None.

In [5]:
test_labels = df['Gold']

predictions = []
for anno in annotations:
    if 'Output' in anno:
        prediction = anno['Output']
    else:
        prediction = "None"
    llama_label = "None"
    for label in test_labels:
        if label in prediction:
            llama_label = label
    predictions.append(llama_label)
print(predictions)
df["LLamaPredictions"]=predictions
df.head(10)

NameError: name 'annotations' is not defined

Note that Llama may not follow the instructions correctly despite the instructions. It may ignore the JSON format, make up new labels or do other "creative" things being triggered by the input text. Always check the output carefully. If Llama does not generate the right output, we can consider this as an error. In this case, it generates "Output:sadness" as a label in one case.

In [ ]:
### We pair the test_labels and the predictions
for pair in zip(test_labels,predictions):
    print(pair)

labels = list(test_labels)+list(predictions)
label_set = sorted(set(labels))
print(label_set)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

report = classification_report(test_labels,predictions ,digits = 7, target_names=label_set)
print('Llama Ekman ----------------------------------------------------------------')

print(report)

We can see that the results are not very good, which we could have guessed from the paired list of values: "joy" is never assigned and "sadness" is assigned when it should not. Think about how to improve the prompt to make it better. 

In [ ]:
print('Confusion matrix SVM')
cf_matrix = confusion_matrix(test_labels,predictions)
print(cf_matrix)
display = ConfusionMatrixDisplay(confusion_matrix=cf_matrix, display_labels=label_set)
display.plot()

## End of notebook